# Contradictory, My Dear Watson

### Detecting contradiction and entailment in multilingual text using TPUs

**Kaggle link:** https://www.kaggle.com/competitions/contradictory-my-dear-watson/overview

### Description

"…when you have eliminated the impossible, whatever remains, however improbable, must be the truth"
-Sir Arthur Conan Doyle

Our brains process the meaning of a sentence like this rather quickly.

We're able to surmise:

Some things to be true: "You can find the right answer through the process of elimination.”
Others that may have truth: "Ideas that are improbable are not impossible!"
And some claims are clearly contradictory: "Things that you have ruled out as impossible are where the truth lies."
Natural language processing (NLP) has grown increasingly elaborate over the past few years. Machine learning models tackle question answering, text extraction, sentence generation, and many other complex tasks. But, can machines determine the relationships between sentences, or is that still left to humans? If NLP can be applied between sentences, this could have profound implications for fact-checking, identifying fake news, analyzing text, and much more.

### The Challenge:
If you have two sentences, there are three ways they could be related: one could entail the other, one could contradict the other, or they could be unrelated. Natural Language Inferencing (NLI) is a popular NLP problem that involves determining how pairs of sentences (consisting of a premise and a hypothesis) are related.

Your task is to create an NLI model that assigns labels of 0, 1, or 2 (corresponding to entailment, neutral, and contradiction) to pairs of premises and hypotheses. To make things more interesting, the train and test set include text in fifteen different languages! You can find more details on the dataset by reviewing the Data page.

Today, the most common approaches to NLI problems include using embeddings and transformers like BERT. In this competition, we’re providing a starter notebook to try your hand at this problem using the power of Tensor Processing Units (TPUs). TPUs are powerful hardware accelerators specialized in deep learning tasks, including Natural Language Processing. Kaggle provides all users TPU Quota at no cost, which you can use to explore this competition. Check out our TPU documentation and Kaggle’s YouTube playlist for more information and resources.

### The Challenge:
If you have two sentences, there are three ways they could be related: one could entail the other, one could contradict the other, or they could be unrelated. Natural Language Inferencing (NLI) is a popular NLP problem that involves determining how pairs of sentences (consisting of a premise and a hypothesis) are related.

Your task is to create an NLI model that assigns labels of 0, 1, or 2 (corresponding to entailment, neutral, and contradiction) to pairs of premises and hypotheses. To make things more interesting, the train and test set include text in fifteen different languages! You can find more details on the dataset by reviewing the Data page.

Today, the most common approaches to NLI problems include using embeddings and transformers like BERT. In this competition, we’re providing a starter notebook to try your hand at this problem using the power of Tensor Processing Units (TPUs). TPUs are powerful hardware accelerators specialized in deep learning tasks, including Natural Language Processing. Kaggle provides all users TPU Quota at no cost, which you can use to explore this competition. Check out our TPU documentation and Kaggle’s YouTube playlist for more information and resources.

### Recommended Tutorial
We highly recommend this excellent tutorial on using KerasNLP to solve this problem, from the Keras team as well as Ana Sofia Uzsoy’s Tutorial that walks you through creating your very first submission step by step with TPUs and BERT.
    This is a great opportunity to flex your NLP muscles and solve an exciting problem!

### Disclaimer:
The dataset for this competition contains text that may be considered profane, vulgar, or offensive.

### Evaluation

#### Goal
Your goal is to predict whether a given hypothesis is related to its premise by contradiction, entailment, or whether neither of those is true (neutral).
For each sample in the test set, you must predict a 0, 1, or 2 value for the variable.

Those values map to the logical condition as:

0 == entailment
1 == neutral
2 == contradiction

#### Metric
Your score is the percentage of relationships you correctly predict. This is known as accuracy.


#### Submission File Format
You should submit a csv file with exactly 5195 entries plus a header row. Your submission will show an error if you have extra columns (beyond id and prediction) or rows.

The file should have exactly 2 columns:

id (sorted in any order)
prediction (contains your predictions: 0 for entailment, 1 for neutral, 2 for contradiction)

### id,prediction

c6d58c3f69,1
cefcc82292,1
e98005252c,1
58518c10ba,1
c32b0d16df,1
Etc.

You can download an example submission file (sample_submission.csv) on the Data page.

### Code Submission Requirement

In this code competition, your submission.csv file must be generated as an output from a Kaggle notebook. For details on how to submit from a notebook, review the FAQ on "How do I make a submission?"

### Dataset Description
In this Getting Started Competition, we’re classifying pairs of sentences (consisting of a premise and a hypothesis) into three categories - entailment, contradiction, or neutral. Let’s take a look at an example of each of these cases for the following premise:

He came, he opened the door and I remember looking back and seeing the expression on his face, and I could tell that he was disappointed.

#### Hypothesis 1:

Just by the look on his face when he came through the door I just knew that he was let down.

We know that this is true based on the information in the premise. So, this pair is related by entailment.


#### Hypothesis 2:

He was trying not to make us feel guilty but we knew we had caused him trouble.

This very well might be true, but we can’t conclude this based on the information in the premise. So, this relationship is neutral.

#### Hypothesis 3:

He was so excited and bursting with joy that he practically knocked the door off it's frame.

We know this isn’t true, because it is the complete opposite of what the premise says. So, this pair is related by contradiction.

This dataset contains premise-hypothesis pairs in fifteen different languages, including:
Arabic, Bulgarian, Chinese, German, Greek, English, Spanish, French, Hindi, Russian, Swahili, Thai, Turkish, Urdu, and Vietnamese.

#### Files:
train.csv: This file contains the ID, premise, hypothesis, and label, as well as the language of the text and its two-letter abbreviation

test.csv: This file contains the ID, premise, hypothesis, language, and language abbreviation, without labels.

sample_submission.csv: This is a sample submission file in the correct format:
id: a unique identifier for each sample
    label: the classification of the relationship between the premise and hypothesis (0 for entailment, 1 for neutral, 2 for contradiction)

Special thanks to Tensorflow Datasets (TFDS) for providing this and many other useful datasets! For more information, visit: https://www.tensorflow.org/datasets

Disclaimer: The dataset for this competition contains text that may be considered profane, vulgar, or offensive.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import *
from sklearn.linear_model import *
from math import *
from nltk import *
import warnings
warnings.filterwarnings("ignore")
from deep_translator import *
import tensorflow as tf
import os
import pathlib as pl

In [2]:
download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/amith/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to
[nltk_data]    |

True

In [3]:
df_train = pd.read_csv("data/train.csv")
df_train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [4]:
df_test = pd.read_csv("data/test.csv")
df_test.head()

,id,premise,hypothesis,lang_abv,language
0,c6d58c3f69,بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...,"کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...",ur,Urdu
1,cefcc82292,هذا هو ما تم نصحنا به.,عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...,ar,Arabic
2,e98005252c,et cela est en grande partie dû au fait que le...,Les mères se droguent.,fr,French
3,58518c10ba,与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp,IMA与其他组织合作，因为它们都依靠共享资金。,zh,Chinese
4,c32b0d16df,Она все еще была там.,"Мы думали, что она ушла, однако, она осталась.",ru,Russian


In [5]:
print("Number of training dataset : ",len(df_train))
print("Number of testing dataset  : ",len(df_test))

Number of training dataset :  12120
Number of testing dataset  :  5195


In [6]:
df_train["lang_abv"].unique()

array(['en', 'fr', 'th', 'tr', 'ur', 'ru', 'bg', 'de', 'ar', 'zh', 'hi',
       'sw', 'vi', 'es', 'el'], dtype=object)

In [7]:
df_train_new = df_train.copy()

Arabic, Bulgarian, Chinese, German, Greek, English, Spanish, French, Hindi, Russian, Swahili, Thai, Turkish, Urdu, and Vietnamese.

In [8]:
french = df_train[df_train["lang_abv"]=="fr"].index.values
thai = df_train[df_train["lang_abv"]=="th"].index.values
turkish = df_train[df_train["lang_abv"]=="tr"].index.values
urdu = df_train[df_train["lang_abv"]=="ur"].index.values
russian = df_train[df_train["lang_abv"]=="ru"].index.values
bulgarian = df_train[df_train["lang_abv"]=="bg"].index.values
german = df_train[df_train["lang_abv"]=="de"].index.values
arabic = df_train[df_train["lang_abv"]=="ar"].index.values
chinese = df_train[df_train["lang_abv"]=="zh"].index.values
hindi = df_train[df_train["lang_abv"]=="hi"].index.values
swahili = df_train[df_train["lang_abv"]=="sw"].index.values
vietnamese = df_train[df_train["lang_abv"]=="vi"].index.values
spanish = df_train[df_train["lang_abv"]=="es"].index.values
greek = df_train[df_train["lang_abv"]=="el"].index.values

In [9]:
def translate_to_english(df,inde):
    tr = GoogleTranslator()
    ctr = 1
    for i in inde:
        df.loc[i,"premise"] = tr.translate(df.loc[i,"premise"])
        df.loc[i,"hypothesis"] = tr.translate(df.loc[i,"hypothesis"])
        print(ctr)
        print("Premise    :  ",df.loc[i,"premise"])
        print("Hypothesis :  ",df.loc[i,"hypothesis"])
        ctr+=1
    return df

In [10]:
df_train_new_1 = translate_to_english(df_train_new,french)

1
Premise    :   Little things like that make a huge difference in what I'm trying to do.
Hypothesis :   I was trying to accomplish something.
2
Premise    :   look at a girl in a white fur parka with boots
Hypothesis :   The girl is dressed all in red.
3
Premise    :   Properly implemented, this approach provides reasonable assurance that a trip has occurred.
Hypothesis :   Approach shows you the journey that has taken place.
4
Premise    :   The complexity of the subjunctive in French does not bother him the least in the world, for the good and simple reason that he does not try to use it.
Hypothesis :   He doesn't try because he has no motivation.
5
Premise    :   Will the White House change its mind?
Hypothesis :   Will the White House change its mind?
6
Premise    :   Notices to the opposition lawyer and to the court or administrative agency should be sent.
Hypothesis :   Opposing counsel and the court expect to receive legal notices.
7
Premise    :   It wouldn't explode without t

In [11]:
df_train_new_2 = translate_to_english(df_train_new_1,thai)

1
Premise    :   In role play as well. Opportunities to express yourself and play multiple roles may help children capture similarities and differences between people in their desires, beliefs and feelings.
Hypothesis :   Children can see how ethnicities are different.
2
Premise    :   On the other hand, in the Cite de la Musique is the Musee de la Musique and its gigantic concert hall, Zenith.
Hypothesis :   Cenine is a large concert hall.
3
Premise    :   Newsweek says tourists and celebrities flock to Patagonia. which was once a safe haven for fugitive Nazis.
Hypothesis :   New hotels have been built in Patagonia to accommodate tourists.
4
Premise    :   when the sand formed The sand then enters the corner to balance. then expands to the edge of the table
Hypothesis :   The sand was scooped by a bulldozer.
5
Premise    :   Title V that performs the request for authorization must be open for public comment.
Hypothesis :   Public comments must be allowed.
6
Premise    :   intuitively 

In [12]:
df_train_new_3 = translate_to_english(df_train_new_2,turkish)

1
Premise    :   You might hear someone on a farm say we have to butcher these oxen in the corral, which probably means, separate the ones to be loaded.
Hypothesis :   On the farm, people use different terms.
2
Premise    :   Others do not make the consumer happy enough.
Hypothesis :   Others are sufficient.
3
Premise    :   Attached to Nova Scotia by the Narrow Chignecto Isthmus, New Brunswick became a separate province in 1784 at the request of 14,000 Loyalist refugees.
Hypothesis :   New Brunswick was a state in 1784.
4
Premise    :   And well, I think we'll be on par with them in terms of pay and reputation in the long run.
Hypothesis :   Ultimately, we should be able to offer similar salary options.
5
Premise    :   This year's alumni luncheon was held during the AMRA Annual Meeting in Nashville, Tennessee, on October 23, 1991.
Hypothesis :   The AMRA Annual Meeting takes place annually in Nashville.
6
Premise    :   Hundreds of excursionists who sail to both islands are willing t

In [13]:
df_train_new_4 = translate_to_english(df_train_new_3,urdu)

1
Premise    :   Upon his return to the United States, Hajj was met at the airport by FBI agents, interrogated, and the next day before a federal grand jury indicted Osama bin Laden.
Hypothesis :   Hague was investigated by FBI agents
2
Premise    :   The most legendary feature of the building is related to the cave arms.
Hypothesis :   There are many weather vanes in the building.
3
Premise    :   what is We have about an acre, yes it's weird because we have
Hypothesis :   It's no joke, we only have one square foot of land.
4
Premise    :   This week's inspiration offers Ken Starr's report on the great star of love, including romance in the presidential limo, the Oval Office and even the liner beddom!
Hypothesis :   The questioner knew about the White House
5
Premise    :   They said they were taken to prison, Jeremy said quietly
Hypothesis :   Jeremy aik saundari dakoo tha apni pori zindagi.
6
Premise    :   The reforms that have been adopted so far have profound effects on what gove

In [14]:
df_train_new_5 = translate_to_english(df_train_new_4,russian)

1
Premise    :   Every hundred degrees, the spots of paint change their color, it can be red and change color to blue.
Hypothesis :   The paint changes according to the color.
2
Premise    :   I assure you, sir, that I was fully aware of everything.
Hypothesis :   Many details were hidden from me.
3
Premise    :   And how much better are these? - Are you afraid of the clumsy Barbados planter? What's bothering you, Peter? I've never seen you scared before. A shot rang out behind them.
Hypothesis :   I've never seen Peter be afraid of anything.
4
Premise    :   In 2003 these codes were cancelled; now all questions relating to international terrorism receive one code - 315.
Hypothesis :   All issues related to terrorism receive the highest priority.
5
Premise    :   We are so accustomed to hearing American companies complain about foreign competition that the accusations made by Kodak after its defeat are taken as just another whining.
Hypothesis :   American companies are responding to f

In [15]:
df_train_new_6 = translate_to_english(df_train_new_5,bulgarian)

1
Premise    :   We may not have had everything we wanted or saw other people have, but she provided the essentials we needed.
Hypothesis :   We had everything we really needed.
2
Premise    :   Left to its own devices, this reaction is exergonic, and in the presence of an excess of trimers, relative to the equilibrium ratio of hexamer to trimers, will proceed exergonically toward equilibrium by synthesizing the hexamer.
Hypothesis :   If this reaction is untouched, it will eventually reach equilibrium.
3
Premise    :   Hope to hear from you soon.
Hypothesis :   Never speak to me again!
4
Premise    :   My hero, however, is Richard Hazeltine, chairman of the Overseas Investment Trust, who resigned earlier this month in disagreement with the business plan imposed by his superiors.
Hypothesis :   Hazeltine quit his job.
5
Premise    :   Relajo can also describe a joking relationship, a mysterious back-and-forth that with laughter relieves the tension and erases the cause of the momentary

In [16]:
df_train_new_7 = translate_to_english(df_train_new_6,german)

1
Premise    :   Twenty percent interest is charged
Hypothesis :   Could the interest be more than 20?
2
Premise    :   Social Security does not contain any established programs exclusively or primarily for federal employees, such as: pension and other retirement plans.
Hypothesis :   Social security plans do not include a plan for federal employees only.
3
Premise    :   The inability to communicate was a critical element at the World Trade Center, Pentagon and Somerset County, Pennsylvania crash sites, where multiple agencies and multiple jurisdictions responded.
Hypothesis :   It was difficult for people in the World Trade Center to communicate.
4
Premise    :   One zip code can be used by multiple routes.
Hypothesis :   Each zip code is served by only one route.
5
Premise    :   This is Fannie Flono, she grew up in Ag--Augusta, GA and is going to share some memories from her childhood.
Hypothesis :   Despite her busy schedule, Fannie Flono was able to speak to us today.
6
Premise  

In [17]:
df_train_new_8 = translate_to_english(df_train_new_7,arabic)

1
Premise    :   If possible, familiarize yourself with the plot in advance.
Hypothesis :   Try to understand the plot first, if you can.
2
Premise    :   But it may be the blood of reason now.
Hypothesis :   A country has quickly come to a conclusion about its position.
3
Premise    :   Well, I'm in Texas and we have a teacher who died of AIDS
Hypothesis :   I have never been to Texas.
4
Premise    :   The spirit of liberalism prevailing in Europe did not reach Spain until late.
Hypothesis :   Spain has never been liberal.
5
Premise    :   The word self-pathological, the word normal rather than the legal word, is a trouble maker and should be avoided.
Hypothesis :   Self-satisfaction is not a problem.
6
Premise    :   In fact, the word quark in the Oxford Dictionary as a verb means 'to pessimise', with 19th century references referring to frogs, crows, and the grouchy king.
Hypothesis :   Most people don't know the origin of the word quark and that it refers to the noise a frog makes.

In [18]:
df_train_new_9 = translate_to_english(df_train_new_8,chinese)

1
Premise    :   I hope your lords will at last begin to realize the folly of granting such a man a Council of Kings against all my advice.
Hypothesis :   Your Excellency has just taken my advice and acted accordingly.
2
Premise    :   The dispute, the management is now pointing to these food stamps, and of course it can also turn to the basic welfare - TANF.
Hypothesis :   The government hasn't said anything about food stamps or benefits.
3
Premise    :   TV dinners bring a bad stain.
Hypothesis :   The people on TV only eat breakfast and lunch.
4
Premise    :   oh i see the vibe here
Hypothesis :   i don't like this style
5
Premise    :   Well, my grandparents were always very, very loving, and so were my parents, and we had a great time down there.
Hypothesis :   My grandparents were a very loving couple.
6
Premise    :   Dubai is a modern city with easy access to major airports, travel agencies, hotels and Western business establishments, making it an ideal stopover.
Hypothesis :  

In [19]:
df_train_new_10 = translate_to_english(df_train_new_9,hindi)

1
Premise    :   Mjodi claims he went home to Morocco to get married but could not make it there due to injuries sustained in a car accident.
Hypothesis :   Mazoudi claimed that he went along with the plan as planned.
2
Premise    :   He looked up in astonishment, and then sat down fuming with her brooding glance.
Hypothesis :   She was braver than he expected.
3
Premise    :   This is no secret to music retailers.
Hypothesis :   Music retailers are completely in the dark about this.
4
Premise    :   well damn! Maybe you will explain yourself? Where has Woverstone gone?
Hypothesis :   I want you to explain yourself! I know you know where Wolverstone is.
5
Premise    :   This thing cannot be said for the New York Times. In its op-ed on the cocaine controversy, the Times advised Bush to be honest and to help the country recover.
Hypothesis :   The Times said that Bush should simply lie to everyone.
6
Premise    :   Population growth is like pollution in reverse.
Hypothesis :   Population

In [20]:
df_train_new_11 = translate_to_english(df_train_new_10,swahili)

1
Premise    :   Children would knock on their neighbors' doors and
Hypothesis :   Children could go to their neighbors house.
2
Premise    :   Such boats were designed to allow access for arriving ships.
Hypothesis :   Boats were introduced to make it easier to catch incoming ships.
3
Premise    :   I have never understood why the international writing of the national phone number is not used in all kinds of English dictionaries, but this is beyond the scope of our words in this review.
Hypothesis :   This review is small in scope so it does not discuss why International Symposium Writing is not used in English dictionaries.
4
Premise    :   On May 1, we will close membership renewal elections for 1991 contributors.
Hypothesis :   There is a chance for 1500 members to renew their membership.
5
Premise    :   He was at the height of the Cuban Missile Crisis and Kaiser got the photos and headed to Andrews Air Force Base in Washington.
Hypothesis :   Only one person was killed in the Cub

In [21]:
df_train_new_12 = translate_to_english(df_train_new_11,vietnamese)

1
Premise    :   Colder and farther than ever has developed the voice of the lord.
Hypothesis :   God's voice feels so distant and cold
2
Premise    :   In the case of US 11, the last normal communication from the plane was at 8:13 am.
Hypothesis :   There has not been any communication from American 11.
3
Premise    :   If anyone had the 1984 edition, he would probably resent having to buy the book rather than a smaller (and less expensive) copy.
Hypothesis :   The surcharge is cheaper than the book.
4
Premise    :   To be sure, the FDNY is not responsible for managing the City's response to the emergency, as the Mayor's directive has required.
Hypothesis :   Management of the City's response was handled by an organization other than the FDNY.
5
Premise    :   Time spent with parents and children together is the first step toward implementing the ideas and practices that I will discuss in this book.
Hypothesis :   In this book, I will cover topics that parents and their children can w

In [22]:
df_train_new_13 = translate_to_english(df_train_new_12,spanish)

1
Premise    :   27 The difficulty increases as the need for boiler modifications to be made to adapt the SCR to the installation also increases.
Hypothesis :   27 The difficulty depends on the modifications.
2
Premise    :   Stevenson's article shows a fundamental lack of understanding of what our campaign entails.
Hypothesis :   Although Stevenson has traveled with us on the campaign, his article makes it clear that he wasn't paying attention to what the campaign is about, even at the most basic level.
3
Premise    :   I told them it was from my sister.
Hypothesis :   I blamed my sister.
4
Premise    :   Ajaj had entered the United States on a B-2 tourist visa in New York City on September 9, 1991.
Hypothesis :   Ajaj was on a tourist visa when he went to the United States.
5
Premise    :   Okay, can you hear me?
Hypothesis :   I know you can't hear me.
6
Premise    :   The Greenlee County, Arizona Public Library illustrates the problem of money and technology for rural institutions.

In [23]:
df_train_new_14 = translate_to_english(df_train_new_13,greek)

1
Premise    :   Consequently, government decision-makers and supervisors adopt new ways of thinking, considering different ways of achieving goals and using new information to guide decisions.
Hypothesis :   Government representatives refuse to change their way of life.
2
Premise    :   With the easiest assimilation into the Anglo community, only Protestant schools accepted their children Eastern European Jews have graduated from wealthy Westmount or immigrated back to Toronto.
Hypothesis :   Jews were more easily integrated into the English community.
3
Premise    :   President Kennedy tells the pilots: Gentlemen, take good pictures.
Hypothesis :   Kennedy did not recognize them.
4
Premise    :   Therefore, I know you go to great lengths to be compassionate and care for others.
Hypothesis :   I know you just don't care about people at all.
5
Premise    :   First Wives Club, a revenge comedy about three jilted wives, earned the most in its opening weekend of any female-led film in his

In [25]:
df_train_new_14

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Little things like that make a huge difference...,I was trying to accomplish something.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,In role play as well. Opportunities to express...,Children can see how ethnicities are different.,th,Thai,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2
